In [4]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [6]:
super_dataset = pd.read_csv('../../../data/stage_I/superDataset.csv')
world_dataset = pd.read_csv('../../../data/stage_II/owid-covid-data.csv')

In [7]:
countries = ['Indonesia', 'Pakistan', 'Nigeria', 'Brazil', 'Bangladesh', 'United States']

filtered_dataset = world_dataset[world_dataset.location.isin(countries)]
filtered_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
5689,BGD,Asia,Bangladesh,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5690,BGD,Asia,Bangladesh,2020-03-04,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5691,BGD,Asia,Bangladesh,2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5692,BGD,Asia,Bangladesh,2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5693,BGD,Asia,Bangladesh,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632


In [8]:
us_dataset = filtered_dataset[filtered_dataset.location == 'United States'].fillna(0)
us_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926


In [9]:
us_dataset[['year', 'week', 'day']] = pd.DataFrame(us_dataset.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d").isocalendar()).to_list(), index=us_dataset.index)
us_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,year,week,day
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,...,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,3
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,...,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,4
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,...,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,5
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,...,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,6
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,...,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,7


In [10]:
# us_dataset['date'] = pd.to_datetime(us_dataset.date)
# us_dataset['week'] = us_dataset.date.dt.isocalendar().week

In [11]:
weekly_data = pd.DataFrame(us_dataset.groupby(['year', 'week']).mean().new_cases.apply(lambda x: math.ceil(x))).reset_index()
weekly_data.head()

,year,week,new_cases
0,2020,4,1
1,2020,5,1
2,2020,6,1
3,2020,7,1
4,2020,8,1


In [12]:
indonesia_dataset = filtered_dataset[filtered_dataset.location == 'Indonesia'].fillna(0)
indonesia_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
30304,IDN,Asia,Indonesia,2020-03-02,2.0,2.0,0.0,0.0,0.0,0.0,...,11188.744,5.7,342.864,6.32,2.8,76.1,64.204,1.04,71.72,0.718
30305,IDN,Asia,Indonesia,2020-03-03,2.0,0.0,0.0,0.0,0.0,0.0,...,11188.744,5.7,342.864,6.32,2.8,76.1,64.204,1.04,71.72,0.718
30306,IDN,Asia,Indonesia,2020-03-04,2.0,0.0,0.0,0.0,0.0,0.0,...,11188.744,5.7,342.864,6.32,2.8,76.1,64.204,1.04,71.72,0.718
30307,IDN,Asia,Indonesia,2020-03-05,2.0,0.0,0.0,0.0,0.0,0.0,...,11188.744,5.7,342.864,6.32,2.8,76.1,64.204,1.04,71.72,0.718
30308,IDN,Asia,Indonesia,2020-03-06,4.0,2.0,0.0,0.0,0.0,0.0,...,11188.744,5.7,342.864,6.32,2.8,76.1,64.204,1.04,71.72,0.718


In [13]:
brazil_dataset = filtered_dataset[filtered_dataset.location == 'Brazil'].fillna(0)
brazil_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
9355,BRA,South America,Brazil,2020-02-26,1.0,1.0,0.0,0.0,0.0,0.0,...,14103.452,3.4,177.961,8.11,10.1,17.9,0.0,2.2,75.88,0.765
9356,BRA,South America,Brazil,2020-02-27,1.0,0.0,0.0,0.0,0.0,0.0,...,14103.452,3.4,177.961,8.11,10.1,17.9,0.0,2.2,75.88,0.765
9357,BRA,South America,Brazil,2020-02-28,1.0,0.0,0.0,0.0,0.0,0.0,...,14103.452,3.4,177.961,8.11,10.1,17.9,0.0,2.2,75.88,0.765
9358,BRA,South America,Brazil,2020-02-29,2.0,1.0,0.0,0.0,0.0,0.0,...,14103.452,3.4,177.961,8.11,10.1,17.9,0.0,2.2,75.88,0.765
9359,BRA,South America,Brazil,2020-03-01,2.0,0.0,0.0,0.0,0.0,0.0,...,14103.452,3.4,177.961,8.11,10.1,17.9,0.0,2.2,75.88,0.765


In [14]:
pakistan_dataset = filtered_dataset[filtered_dataset.location == 'Pakistan'].fillna(0)
pakistan_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
49794,PAK,Asia,Pakistan,2020-02-25,2.0,2.0,0.0,0.0,0.0,0.0,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
49795,PAK,Asia,Pakistan,2020-02-26,2.0,0.0,0.0,0.0,0.0,0.0,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
49796,PAK,Asia,Pakistan,2020-02-27,2.0,0.0,0.0,0.0,0.0,0.0,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
49797,PAK,Asia,Pakistan,2020-02-28,4.0,2.0,0.0,0.0,0.0,0.0,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
49798,PAK,Asia,Pakistan,2020-02-29,4.0,0.0,0.0,0.0,0.0,0.0,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557


In [15]:
nigeria_dataset = filtered_dataset[filtered_dataset.location == 'Nigeria'].fillna(0)
nigeria_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
47477,NGA,Africa,Nigeria,2020-02-28,1.0,1.0,0.0,0.0,0.0,0.0,...,5338.454,0.0,181.013,2.42,0.6,10.8,41.949,0.0,54.69,0.539
47478,NGA,Africa,Nigeria,2020-02-29,1.0,0.0,0.0,0.0,0.0,0.0,...,5338.454,0.0,181.013,2.42,0.6,10.8,41.949,0.0,54.69,0.539
47479,NGA,Africa,Nigeria,2020-03-01,1.0,0.0,0.0,0.0,0.0,0.0,...,5338.454,0.0,181.013,2.42,0.6,10.8,41.949,0.0,54.69,0.539
47480,NGA,Africa,Nigeria,2020-03-02,1.0,0.0,0.0,0.0,0.0,0.0,...,5338.454,0.0,181.013,2.42,0.6,10.8,41.949,0.0,54.69,0.539
47481,NGA,Africa,Nigeria,2020-03-03,1.0,0.0,0.0,0.0,0.0,0.0,...,5338.454,0.0,181.013,2.42,0.6,10.8,41.949,0.0,54.69,0.539


In [16]:
bangladesh_dataset = filtered_dataset[filtered_dataset.location == 'Bangladesh'].fillna(0)
bangladesh_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
5689,BGD,Asia,Bangladesh,2020-03-03,0.0,0.0,0.0,0.0,0.0,0.0,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5690,BGD,Asia,Bangladesh,2020-03-04,0.0,0.0,0.0,0.0,0.0,0.0,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5691,BGD,Asia,Bangladesh,2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5692,BGD,Asia,Bangladesh,2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5693,BGD,Asia,Bangladesh,2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632


In [17]:
main_table = filtered_dataset.fillna(0)

In [18]:
all_cases = main_table[['location','date','new_cases','population']]

In [19]:
all_cases

,location,date,new_cases,population
5689,Bangladesh,2020-03-03,0.0,164689383.0
5690,Bangladesh,2020-03-04,0.0,164689383.0
5691,Bangladesh,2020-03-05,0.0,164689383.0
5692,Bangladesh,2020-03-06,0.0,164689383.0
5693,Bangladesh,2020-03-07,0.0,164689383.0
...,...,...,...,...
69377,United States,2021-02-28,51204.0,331002647.0
69378,United States,2021-03-01,58765.0,331002647.0
69379,United States,2021-03-02,56758.0,331002647.0
69380,United States,2021-03-03,65909.0,331002647.0


In [20]:
all_cases['norm']=(all_cases.new_cases/all_cases.population)*1000000

<ipython-input-20-1de522d8ecda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cases['norm']=(all_cases.new_cases/all_cases.population)*1000000


In [21]:
all_cases['log'] = np.log(all_cases['norm']).replace(-np.inf,0)

/Users/Erika/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-21-7dd6be3b6e0d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cases['log'] = np.log(all_cases['norm']).replace(-np.inf,0)


In [25]:
all_cases

,location,date,new_cases,population,norm,log
5689,Bangladesh,2020-03-03,0.0,164689383.0,0.000000,0.000000
5690,Bangladesh,2020-03-04,0.0,164689383.0,0.000000,0.000000
5691,Bangladesh,2020-03-05,0.0,164689383.0,0.000000,0.000000
5692,Bangladesh,2020-03-06,0.0,164689383.0,0.000000,0.000000
5693,Bangladesh,2020-03-07,0.0,164689383.0,0.000000,0.000000
...,...,...,...,...,...,...
69377,United States,2021-02-28,51204.0,331002647.0,154.693627,5.041447
69378,United States,2021-03-01,58765.0,331002647.0,177.536345,5.179175
69379,United States,2021-03-02,56758.0,331002647.0,171.472949,5.144426
69380,United States,2021-03-03,65909.0,331002647.0,199.119254,5.293904


In [30]:
us_log_dataset = all_cases[all_cases['location'] == 'United States']
us_log_dataset

,location,date,new_cases,population,norm,log
68974,United States,2020-01-22,0.0,331002647.0,0.000000,0.000000
68975,United States,2020-01-23,0.0,331002647.0,0.000000,0.000000
68976,United States,2020-01-24,1.0,331002647.0,0.003021,-5.802126
68977,United States,2020-01-25,0.0,331002647.0,0.000000,0.000000
68978,United States,2020-01-26,3.0,331002647.0,0.009063,-4.703514
...,...,...,...,...,...,...
69377,United States,2021-02-28,51204.0,331002647.0,154.693627,5.041447
69378,United States,2021-03-01,58765.0,331002647.0,177.536345,5.179175
69379,United States,2021-03-02,56758.0,331002647.0,171.472949,5.144426
69380,United States,2021-03-03,65909.0,331002647.0,199.119254,5.293904


In [31]:
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import Image

fig = go.Figure()
fig.add_trace(go.Scatter(x=us_log_dataset["date"], y=us_log_dataset["log"],
                    mode='lines+markers',
                    name='Oregon'))
fig.show()

#all_cases.plotly(x='date', y='log')

In [ ]:
import pandas as pd

ex1 = indonesia_dataset.plot(x='date', y='log')
brazil_dataset.plot(x='date', y='log', ax=ex1)
indonesia_dataset.plot(x='date', y='log', ax=ex1)
pakistan_dataset.plot(x='date', y='log', ax=ex1)
nigeria_dataset.plot(x='date', y='log', ax=ex1)
bangladesh_dataset.plot(x='date', y='log', ax=ex1)
us_dataset.plot(x='date', y='log', ax=ex1)
ex1.legend(["Indonesia", "Brazil", "US", "Pakistan", "Nigeria", "Bangladesh"])

In [ ]:
all_deaths = main_table[['location','date','new_deaths','population']] 

In [ ]:
all_deaths

In [ ]:
all_deaths['norm']=(all_deaths.new_deaths/all_deaths.population)*1000000

In [ ]:
all_deaths['log'] = np.log(all_deaths['norm']).replace(-np.inf,0)

In [ ]:
all_deaths

In [ ]:
import pandas as pd

all_deaths.plot(x='date', y='log')